In [29]:
import pyarrow as pa

In [30]:
hdfs = pa.hdfs.HadoopFileSystem(host="10.3.0.2", port=8020)

In [31]:
hdfs.ls('/user/swapanc1')

['/user/swapanc1/homework1_images']

## Downloading the Csv Files using HDFS.

In [32]:
hdfs.download('/data/google_open_image/ids/test-images-with-rotation.csv', stream = 'test-images-with-rotation.csv', buffer_size=None)
hdfs.download('/data/google_open_image/labels/test-annotations-human-imagelabels-boxable.csv', stream = 'test-annotations-human-imagelabels-boxable.csv', buffer_size=None)
hdfs.download('/data/google_open_image/metadata/class-descriptions-boxable.csv', stream = 'class-descriptions-boxable.csv', buffer_size=None)

## Step 2: Creating separate Dataframes for all the 3 CSV Files.

In [33]:
from pyarrow import csv
import pandas as pd
file1 = 'test-images-with-rotation.csv'
file2='test-annotations-human-imagelabels-boxable.csv'
file3='class-descriptions-boxable.csv'
df1=pd.read_csv(file1)
df2=pd.read_csv(file2)
df3=pd.read_csv(file3)

FileNotFoundError: File b'test-annotations-human-imagelabels-boxable1.csv' does not exist

In [ ]:
df1.head(2)

In [ ]:
df1.shape

In [ ]:
df2.head(2)

In [ ]:
df2.shape

In [ ]:
df3.head(2)

## The 3rd CSV File contains No column Name, hence adding Column Names called Label and Item and the adding the deleted value back to the Data Frame.

In [ ]:
df3.columns =['Label', 'Item']
df3.head(2)

In [ ]:
df3.loc['0',:]=['/m/011k07','Tortoise']  
df3.tail(2)

In [ ]:
df3.shape

## Use the join command to join the three dataframes into a single dataframe. Ensure you don't have repeated lines in the resulting dataframe. File 3 can be joined with file 2 based on the Label field. (File 2 calls this LabelName, File 3 calls it Label) That dataframe can be joined with File 1 based on the Image ID field.

In [ ]:
join_df = df2.join(df3.set_index('Label'), on='LabelName')
join_df.head(15)

In [ ]:
join_df.shape

In [ ]:
join_df.tail()

In [ ]:
join_df2=join_df.join(df1.set_index('ImageID'), on='ImageID')
join_df2.head(2)

In [ ]:
join_df2.shape

In [ ]:
join_df2.iloc[0,6]

In [ ]:
join_df2.head()

## Step 3: Download Images

In [ ]:
import requests
response = requests. get(join_df2.iloc[5,6])
file = open("image.jpg", "wb")
file. write(response. content)
file.close()

In [ ]:
response = requests. get(join_df2.iloc[7,6])
file = open("image1.jpg", "wb")
file. write(response. content)
file.close()

In [ ]:
response = requests. get(join_df2.iloc[16,6])
file = open("image2.jpg", "wb")
file. write(response. content)
file.close()

## Creaing a New Directory using mkdir

In [ ]:
!hdfs dfs -mkdir /user/swapanc1/homework1_images

## Step 4: Uploading the downloaded Images to HDFS

In [ ]:
!hadoop fs -copyFromLocal 'image.jpg' '/user/swapanc1/homework1_images'
!hadoop fs -copyFromLocal 'image1.jpg' '/user/swapanc1/homework1_images'
!hadoop fs -copyFromLocal 'image2.jpg' '/user/swapanc1/homework1_images'

In [ ]:
hdfs.ls('/user/swapanc1/homework1_images') ## Checking the uploaded files in the directory.